### Librerias:

In [1]:
! pip install spacy langchain azure-search-documents azure-core transformers tf-keras torch




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
! python3 -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import json
import spacy
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from transformers import pipeline

In [13]:
from dotenv import load_dotenv, find_dotenv
import os

# Cargar archivo .env
dotenv_path = find_dotenv()
if dotenv_path:
    print(f"Archivo .env encontrado en: {dotenv_path}")
    load_dotenv(dotenv_path)
else:
    print("Archivo .env no encontrado. Asegúrate de que exista y esté en la ruta correcta.")

# Variables necesarias para Azure Cognitive Search
AZURE_SEARCH_ENDPOINT = os.getenv("SEARCH_SERVICE_ENDPOINT", "").strip()
AZURE_SEARCH_KEY = os.getenv("SEARCH_API_KEY", "").strip()
INDEX_NAME = os.getenv("INDEX_NAME", "").strip()



Archivo .env encontrado en: /Users/luisalbertocerelli/Desktop/00-Todo/14_Hackaton_Real/PRINCIPAL/02_Uploading_Azure/.env


In [5]:
# Cliente de Azure Search
search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)


In [6]:
# Carga modelo NLP
nlp = spacy.load("en_core_web_sm")
factual_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", framework="pt")



Device set to use mps:0


In [7]:
# Función para verificar hechos

def verificar_factualidad(texto, documentos_referencia):
    """Verifica la factualidad del texto usando documentos de referencia."""
    respuestas = []
    for doc in documentos_referencia:
        respuesta = factual_pipeline({
            "question": texto,
            "context": doc
        })
        respuestas.append(respuesta["score"])
    return max(respuestas) if respuestas else 0

In [8]:
# Búsqueda en Azure

def buscar_en_azure(query):
    """Busca información relevante en Azure Search."""
    resultados = search_client.search(query)
    documentos = [r["content"] for r in resultados]
    return documentos

In [9]:
# Análisis de consistencia

def analizar_coherencia(texto):
    """Analiza la coherencia interna del texto."""
    doc = nlp(texto)
    inconsistencias = []
    for sent in doc.sents:
        tokens = [token.text for token in sent if token.dep_ == "neg"]
        if tokens:
            inconsistencias.append(sent.text)
    return inconsistencias

In [10]:
# Orquestador de LangChain

def evaluar_prompt(texto):
    """Evalúa un texto generado en busca de alucinaciones."""
    # Buscar documentos de referencia
    documentos = buscar_en_azure(texto)
    if not documentos:
        return {
            "alucinaciones_probabilidad": 1.0,
            "comentarios": "No se encontraron documentos de referencia para el texto.",
            "acciones_recomendadas": ["Agregar documentos relevantes al índice Azure."]
        }

    # Verificar factualidad
    factualidad = verificar_factualidad(texto, documentos)

    # Analizar coherencia
    inconsistencias = analizar_coherencia(texto)

    # Generar reporte
    probabilidad_alucinacion = 1 - factualidad if factualidad else 1.0
    comentarios = []

    if probabilidad_alucinacion > 0.5:
        comentarios.append("La información parece ficticia o no respaldada por los documentos.")
    if inconsistencias:
        comentarios.append(f"Inconsistencias detectadas: {', '.join(inconsistencias)}")

    return {
        "alucinaciones_probabilidad": probabilidad_alucinacion,
        "comentarios": " ".join(comentarios),
        "acciones_recomendadas": [
            "Verificar el contenido manualmente.",
            "Agregar más datos relevantes al índice de Azure."
        ]
    }




In [11]:
# Prueba de ejemplo
if __name__ == "__main__":
    texto_ejemplo = "La huella de carbono mide emisiones de CO₂ producidas por actividades humanas."
    resultado = evaluar_prompt(texto_ejemplo)
    print(json.dumps(resultado, indent=4))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{
    "alucinaciones_probabilidad": 0.4282378554344177,
    "comentarios": "",
    "acciones_recomendadas": [
        "Verificar el contenido manualmente.",
        "Agregar m\u00e1s datos relevantes al \u00edndice de Azure."
    ]
}
